In [2]:
from functools import reduce

import matplotlib.pyplot as plt
import numpy as np
import qutip as qt
from quantum_logical import Pulse, DressedQuantumSystem
from quantum_logical.hamiltonian import QubitSNAILModule
from quantum_logical.mode import QubitMode, SNAILMode
from qutip import Options
from multiprocessing.pool import ThreadPool

from tqdm import tqdm
from scipy.optimize import minimize
import scienceplots


opts = {"nsteps": 1e6, "atol": 1e-9, "rtol": 1e-7}  # , "progress_bar": "text"}

In [3]:
freqs = np.array([5.06167, 4.54944, 4.16829])  # q1, q2, q3 frequencies
snail_freq = 4.27515  # SNAIL frequency

In [4]:
a = 5.06167
c = 4.16829
b = 4.54944
s = 4.27515
center = a - ((a - b) / 2)
def get_array(a, b, c, s, option, divisions):
    array = []
    array.clear()
    if option == 'scale center':
        center_freq = a - ((a - b) / 2)
        for i in range(divisions):
            new_a = a - (i/(divisions - 1)) * (a - center_freq)
            new_b = b + (i/(divisions - 1)) * np.abs(b - center_freq)
            new_c = c + (i/(divisions - 1)) * np.abs(c - center_freq)
            new_s = s + (i/(divisions - 1)) * np.abs(s - center_freq)
            array.append([new_a, new_b, new_c, new_s])
    elif option == 'snail scan':
        snail = np.linspace(a, c, divisions)
        for s in snail:
            array.append([a, b, c, s])
    elif option == 'ratio conservation':
        das = a - s
        dbs = b - s
        dcs = c - s
        per = np.linspace(0, 1, divisions)
        for p in per:
            a_new = (p * das) + s
            b_new = (p * dbs) + s
            c_new = (p * dcs) + s
            array.append([a_new, b_new, c_new, s])
    return array

In [6]:
freqs = get_array(a = a, b = b, c = c, s = s, option='scale center', divisions=15)
freqs[0]

[5.06167, 4.54944, 4.16829, 4.27515]

In [19]:
def test_freq(freqs):
    qubit_dim = 2
    qubit1 = QubitMode(name="q1", dim=qubit_dim, freq=freqs[0])
    qubit2 = QubitMode(name="q2", dim=qubit_dim, freq=freqs[1])
    qubit3 = QubitMode(name="q3", dim=qubit_dim, freq=freqs[2])
    qubits = [qubit1, qubit2, qubit3]
    snail = SNAILMode(name="s", freq=freqs[3], g3=0.3, dim=10, T1=1e3, T2=5e2)

    # define couplings so hybridizations are all equal
    # g/delta = 0.1 for all qubits
    g2_0 = 0.1 * np.abs(snail.freq - qubit1.freq)
    g2_1 = 0.1 * np.abs(snail.freq - qubit2.freq)
    g2_2 = 0.1 * np.abs(snail.freq - qubit3.freq)
    _couplings = {
        frozenset([qubit1, snail]): g2_0,
        frozenset([qubit2, snail]): g2_1,
        frozenset([qubit3, snail]): g2_2,
    }

    qs = DressedQuantumSystem(
    qubits + [snail], couplings=_couplings, hamiltonian_cls=QubitSNAILModule)


    # (undressed) expectation operators
    e_ops = [qs.modes_num[m] for m in qs.modes]

    # collapse operators
    c_ops = []
    # for mode in qs.modes:
    #    c_ops.append(mode.collapse_operators(qs))

    # create an initial state
    mode_states = tuple([(qubit1, 0), (qubit2, 1), (qubit3, 0)])
    psi0_a = qs.prepare_approx_state(mode_states)
    # mode_states = tuple([(qubit1, 0), (qubit2, 0), (qubit3, 1)])
    # psi0_b = qs.prepare_approx_state(mode_states)
    # psi0 = (psi0_a + psi0_b).unit()
    psi0 = psi0_a.unit()
    rho0 = psi0 * psi0.dag()

    from qutip_qip.operations import iswap

    desired_U = iswap()  # The iSWAP gate for a 2-qubit system

    # Create isometries for qubit 1 and qubit 2 to extend the {g, e} subspace action to the full qubit space
    identity_isometry = (
        qt.basis(qubit_dim, 0) * qt.basis(2, 0).dag()
        + qt.basis(qubit_dim, 1) * qt.basis(2, 1).dag()
    )
    identity_isometry = qt.tensor(identity_isometry, identity_isometry)

    # Apply the isometry to extend the gate action to the complete system
    extended_q1_q2 = identity_isometry * desired_U * identity_isometry.dag()

    # Tensor with identity matrices for the remaining qubits and the SNAIL mode
    # Skip the first two qubits as they're already included
    # Skip the last index as it's the SNAIL mode
    for mode in qs.modes[2:-1]:
        extended_q1_q2 = qt.tensor(extended_q1_q2, qt.qeye(mode.dim))

    # The extended_iswap_q1_q2 now acts as the desired iSWAP gate on {g, e} of qubits 1 and 2, and as identity on the rest
    desired_U = extended_q1_q2

    # act on qubit space only
    qubit_rho0 = rho0.ptrace(range(len(qubits)))
    expected_qubit_rho = qt.Qobj(desired_U * qubit_rho0 * desired_U.dag())

    width_d = 300  # ns
    off_d = 20
    # args = {"shape": Pulse.box, "shape_params": {"t0": off_d, "width": width_d}}
    args = {"shape": Pulse.smoothbox, "shape_params": {"t0": off_d, "width": width_d}}
    full_time = np.linspace(0, width_d + 2 * off_d, 500)
    wp = np.abs(qubit1.freq - qubit2.freq)
    pulse = Pulse(omega=wp, amp=6.5)

    # Plot the Gaussian pulse shape
    # Pulse.plot_pulse([(pulse, args)], full_time)

    # single period Pulse
    wp = np.abs(qubit1.freq - qubit2.freq)
    T = 2 * np.pi / wp
    period_time = np.linspace(0, T, 250)  # a single period of the pulse
    args = {"shape": Pulse.constant}
    pulse = Pulse(omega=wp, amp=6.5)
    # Pulse.plot_pulse([(pulse, args)], period_time)

    def mesolve_task(omega_amp_tuple):
        omega, amp = omega_amp_tuple
        pulse = Pulse(omega=omega, amp=amp)
        args = {"shape": Pulse.smoothbox, "shape_params": {"t0": off_d, "width": width_d}}
        H_pump = qs.hamiltonian.driven_term(snail_mode=snail)
        H = [qs.hamiltonian.H0, [H_pump, pulse.drive]]
        result = qt.mesolve(H, psi0, full_time, c_ops, options=opts, args=args)
        return [(t, state) for t, state in zip(result.times, result.states)]


    def _construct_propagator(omega_amp_tuple):
        omega, amp = omega_amp_tuple
        T = 2 * np.pi / omega
        pulse = Pulse(omega=omega, amp=amp)
        H_pump = qs.hamiltonian.driven_term(snail_mode=snail)
        H = [qs.hamiltonian.H0, [H_pump, pulse.drive]]
        U_t = qt.propagator(H, T, c_ops, args=args, options=opts)

        # off pump
        pulse_off = Pulse(omega=omega, amp=0)
        H_off = [qs.hamiltonian.H0, [H_pump, pulse_off.drive]]
        U_off = qt.propagator(H_off, off_d, c_ops, args=args, options=opts)
        return U_t, U_off


    def _construct_full_propagator(omega_amp_tuple):
        # create total time evolution operator
        omega, amp = omega_amp_tuple
        T = 2 * np.pi / omega
        U_t, U_off = _construct_propagator(omega_amp_tuple)
        n_periods = int(width_d / T)
        U_total = U_off * (U_t**n_periods) * U_off
        return U_total, full_time[-1]


    def propagator_task(omega_amp_tuple) -> qt.Qobj:
        omega, amp = omega_amp_tuple
        T = 2 * np.pi / omega
        n_periods = int(width_d / T)
        U_t, U_off = _construct_propagator((omega, amp))
        states = [rho0]
        times = [0]

        # initial off pulse
        rho_tt = qt.Qobj(U_off * rho0 * U_off.dag())
        states.append(rho_tt)
        times.append(off_d)

        for _ in range(n_periods):
            rho_tt = qt.Qobj(U_t * rho_tt * U_t.dag())
            states.append(rho_tt)
            times.append(times[-1] + T)

        # final off pulse
        rho_tt = qt.Qobj(U_off * rho_tt * U_off.dag())
        states.append(rho_tt)
        times.append(times[-1] + off_d)

        return [(t, state) for t, state in zip(times, states)]


    def plot_expected_occupations(result_obj):
        # Prepare plot
        plt.figure(figsize=(6, 6))

        times = [t for t, _ in result_obj]

        # Collect data for all modes
        for mode in qs.modes_num:
            populations = [
                np.abs(qt.expect(qs.modes_num[mode], state)) for _, state in result_obj
            ]
            # Plot each mode's population over time
            plt.plot(times, populations, label=mode.name, marker="o")

        # Setting plot labels and title
        plt.xlabel("Time (ns)")
        plt.ylabel("Expected Population")
        plt.title("Expected Population of Modes Over Time")
        plt.legend()
        plt.grid(True)

        # Show plot
        plt.show()


    # final state fidelity
    def extract_state_fidelity(result_obj):
        final_state = result_obj[-1][1]
        qubit_rhof = final_state.ptrace(range(len(qubits)))
        return qt.fidelity(qubit_rhof, expected_qubit_rho)
    
    # create the optimizer and then run the result of the optimizer in the bar below
    detuning = np.linspace(0, .2, 20)
    amp = np.linspace(8, 16, 20) 
    amp_detuning = [(i, j) for i in detuning for j in amp]
    fids = []
    fids.clear()

    def sim_task(amp_detuning):
        detuning, amp = amp_detuning

        me_result = mesolve_task((wp + detuning, amp))
        fid = extract_state_fidelity(me_result)

        fids.append(fid)
        return [detuning, amp, fid]
    
    results = qt.parallel.parallel_map(sim_task, amp_detuning, progress_bar=True)
        

    # # amplitude = results.index(results.max(axis = 0)[2])[1]
    # # detuned = results.index(results.max(axis = 0)[2])[0]

    # res = []
    # for i in tqdm(range(len(amp_detuning))):
    #     detuning, amp = amp_detuning[i]

    #     me_result = mesolve_task((wp + detuning, amp))
    #     fid = extract_state_fidelity(me_result)

    #     fids.append(fid)

    #     res.append([detuning, amp, fid])

    # data analysis from above
    max = 0.0
    for i in range(len(results)):
        fid = results[i][2]
        if fid > max:
            amplitude = results[i][1]
            detune = results[i][0]

    # detuning = (25/3) * 2 * np.pi / 1000
    # amplitude = (125/9)
    me_result = mesolve_task((wp + detune, amplitude))
    # plot_expected_occupations(me_result)
    return extract_state_fidelity(me_result)
    # this will need to be optimized because right now it does not make sense that this can be done over it 

In [13]:
fids = []
fids.clear()
for i in range(len(freqs) - 1):
    fids.append(test_freq(freqs[i]))

Found overlap with eigenstate by 0.9886
Found overlap with eigenstate by 0.9881
Found overlap with eigenstate by 0.9876
Found overlap with eigenstate by 0.9868
Found overlap with eigenstate by 0.9857
Found overlap with eigenstate by 0.9843
Found overlap with eigenstate by 0.9821
Found overlap with eigenstate by 0.9785
Found overlap with eigenstate by 0.9717
Found overlap with eigenstate by 0.9554
Found overlap with eigenstate by 0.8934
Found overlap with eigenstate by 0.8436
Found overlap with eigenstate by 0.9857
Found overlap with eigenstate by 0.9982
Found overlap with eigenstate by 0.9994


C:\Users\girgi\AppData\Local\Temp\ipykernel_31748\3817406093.py:81: RuntimeWarning: divide by zero encountered in scalar divide
  T = 2 * np.pi / wp
c:\Users\girgi\Desktop\Github\quantum_logical\venv\lib\site-packages\numpy\core\function_base.py:158: RuntimeWarning: invalid value encountered in multiply
  y *= step
